In [1]:
import os
import pandas as pd
from tqdm import tqdm
import re

In [2]:
data_path='/opt/ml/book/data'

In [3]:
concat_data=pd.DataFrame()
# open all csv files in data_path
for data in tqdm(os.listdir(data_path)):
    if data.endswith('.csv'):
        df = pd.read_csv(os.path.join(data_path, data),low_memory=False,error_bad_lines=False)
        # we only need the title and the description
        df=df[['title','description']]
        # drop na values
        df.dropna(inplace=True)
        concat_data=concat_data.append(df)
len(concat_data)
        

100%|██████████| 15/15 [01:19<00:00,  5.30s/it]


4256189

In [4]:
concat_data.reset_index(inplace=True)
hangeul_list=[]

# remove all non-hangeul titles
for index in tqdm(concat_data.index):
    title=concat_data.loc[index,'title']
    hangeul=re.findall(f'[가-힣]',title)
    if hangeul:
        hangeul_list.append(index)
only_hangeul=concat_data.loc[hangeul_list]

100%|██████████| 4256189/4256189 [01:30<00:00, 46785.47it/s]


In [5]:
only_hangeul=only_hangeul[['title','description']]

In [6]:
len(only_hangeul)

3565121

In [7]:
# drop title duplicates 
only_hangeul.drop_duplicates(subset=['title'],inplace=True)
len(only_hangeul)

1256761

In [8]:
# drop description duplicates -> 시리즈 같은 것들 제거
only_hangeul.drop_duplicates(subset=['description'],inplace=True)
len(only_hangeul)

996513

In [9]:
# reset index
only_hangeul.reset_index(inplace=True,drop=True)
only_hangeul

,title,description
0,팔꿈치를 주세요,‘큐큐퀴어단편선’의 네 번째 책으로 한국문학의 틀에서 벗어나 자신만의 소설적 세계를...
1,"너처럼 예쁜 동시, 나태주 동시 따라 쓰기",풀꽃 시인이 어린이들에게 전하는 다정한 동시들!차분차분 시를 읽고 따라 쓰다 보면 ...
2,곤고한 날에는 생각하라 - 한 신학자의 인문 고전 읽기,“성서와 인문 고전을 씨줄과 날줄 삼아 직조해 온 치열한 사유와 실천의 기록!”김진...
3,텐동의 사연과 나폴리탄의 비밀,일본인 친구가 한국어로 쓴 음식여행 안내서. 두 번째 이야기. 우리가 좋아하는 일...
4,구름할머니의 콧구멍,구름이는 낮잠 주무시는 할머니 콧구멍 속을 들락날락 숨바꼭질하는 걸 제일 좋아한다....
...,...,...
996508,과학기술 영작문의 기본,"이 책은 종래의 이공계 영어작문서들과는 달리 틀리기 쉬운 단어나 문장을 쓰는 방법,..."
996509,(향가와 옛 한지의 으늑함)옛빛찾기=(The)search for the archai...,"1956년 출생해 원광대 서예과 교수를 지내고 있는 저 자의 작품집. 붓글씨, 동양..."
996510,통일신라 토지제도 연구,통일신라시대 토지제도에 대해 고찰한 연구서. 골품귀족과 국가기관에 대한 토지분급제의...
996511,"민법요해 2 - 債權 親族 相續法, 제2판",- 세부 독자대상 : 사법시험 준비생- 특징 : 사법시험 1차와 2차를 동시에 대비...


In [10]:
#전처리 과정

#이상한 문자들 제거
only_hangeul['description']=only_hangeul['description'].str.replace(pat='&#x0D;',repl='')
only_hangeul['description']=only_hangeul['description'].str.replace(pat='&lt;',repl='')
only_hangeul['description']=only_hangeul['description'].str.replace(pat='&gt;',repl='')

# 필요 없는 특수문자 제거
only_hangeul['description']=only_hangeul['description'].str.replace(pat=r'[「」『』《》&#*+@$%^%;▶★♥◆]',repl=r'',regex=True)

# 이상한 따옴표 변환
only_hangeul['description']=only_hangeul['description'].str.replace(pat=r'[“”]',repl=r'"',regex=True)
only_hangeul['description']=only_hangeul['description'].str.replace(pat=r'[‘’]',repl=r"'",regex=True)

# 앞에 붙은 [] 문자 제가
only_hangeul['description']=only_hangeul['description'].str.replace(pat=r'\[머라말\]',repl=r"",regex=True)
only_hangeul['description']=only_hangeul['description'].str.replace(pat=r'\[서문\]',repl=r"",regex=True)
only_hangeul['description']=only_hangeul['description'].str.replace(pat=r'\[표지글\]',repl=r"",regex=True)
only_hangeul['description']=only_hangeul['description'].str.replace(pat=r'\[표지글 중에서\]',repl=r"",regex=True)

# 중앙 점 ,로 변경
only_hangeul['description']=only_hangeul['description'].str.replace(pat=r'ㆍ',repl=r",",regex=True)

In [11]:
only_hangeul.head()

,title,description
0,팔꿈치를 주세요,'큐큐퀴어단편선'의 네 번째 책으로 한국문학의 틀에서 벗어나 자신만의 소설적 세계를...
1,"너처럼 예쁜 동시, 나태주 동시 따라 쓰기",풀꽃 시인이 어린이들에게 전하는 다정한 동시들!차분차분 시를 읽고 따라 쓰다 보면 ...
2,곤고한 날에는 생각하라 - 한 신학자의 인문 고전 읽기,"""성서와 인문 고전을 씨줄과 날줄 삼아 직조해 온 치열한 사유와 실천의 기록!""김진..."
3,텐동의 사연과 나폴리탄의 비밀,일본인 친구가 한국어로 쓴 음식여행 안내서. 두 번째 이야기. 우리가 좋아하는 일...
4,구름할머니의 콧구멍,구름이는 낮잠 주무시는 할머니 콧구멍 속을 들락날락 숨바꼭질하는 걸 제일 좋아한다....


In [12]:
# save dataframe
only_hangeul.to_csv(os.path.join(data_path,'all_data.csv'),index=False,encoding='utf-8')

In [13]:
# 잘 저장됐는지 확인
a=pd.read_csv('/opt/ml/book/data/all_data.csv')
a.head()

,title,description
0,팔꿈치를 주세요,'큐큐퀴어단편선'의 네 번째 책으로 한국문학의 틀에서 벗어나 자신만의 소설적 세계를...
1,"너처럼 예쁜 동시, 나태주 동시 따라 쓰기",풀꽃 시인이 어린이들에게 전하는 다정한 동시들!차분차분 시를 읽고 따라 쓰다 보면 ...
2,곤고한 날에는 생각하라 - 한 신학자의 인문 고전 읽기,"""성서와 인문 고전을 씨줄과 날줄 삼아 직조해 온 치열한 사유와 실천의 기록!""김진..."
3,텐동의 사연과 나폴리탄의 비밀,일본인 친구가 한국어로 쓴 음식여행 안내서. 두 번째 이야기. 우리가 좋아하는 일...
4,구름할머니의 콧구멍,구름이는 낮잠 주무시는 할머니 콧구멍 속을 들락날락 숨바꼭질하는 걸 제일 좋아한다....
